# GRU

RNN存在的问题：梯度较容易出现衰减或爆炸（BPTT）  
⻔控循环神经⽹络：捕捉时间序列中时间步距离较大的依赖关系  
**RNN**:  


![Image Name](https://cdn.kesci.com/upload/image/q5jjvcykud.png?imageView2/0/w/320/h/320)


$$
H_{t} = ϕ(X_{t}W_{xh} + H_{t-1}W_{hh} + b_{h})
$$
**GRU**:


![Image Name](https://cdn.kesci.com/upload/image/q5jk0q9suq.png?imageView2/0/w/640/h/640)



$$
R_{t} = σ(X_tW_{xr} + H_{t−1}W_{hr} + b_r)\\    
Z_{t} = σ(X_tW_{xz} + H_{t−1}W_{hz} + b_z)\\  
\widetilde{H}_t = tanh(X_tW_{xh} + (R_t ⊙H_{t−1})W_{hh} + b_h)\\
H_t = Z_t⊙H_{t−1} + (1−Z_t)⊙\widetilde{H}_t
$$
• 重置门有助于捕捉时间序列⾥短期的依赖关系；  
• 更新门有助于捕捉时间序列⾥⻓期的依赖关系。 

原理：https://zhuanlan.zhihu.com/p/78456130  


    对于重置门R：如果R趋于0，那么意味着在H_tilde的计算中将H_(t-1)清零；
            如果R趋于1，那么意味着在H_tilde的计算中保留原来的H_(t-1)；
            通过这样达到重置的效果，进而捕捉短期的依赖关系。
            

    对于更新门Z：如果Z趋于0，那么在H_t的计算中将H_t计算为H_tilde；
            如果Z趋于1，那么在H_t的计算中将H_t计算为H_(t-1)，与上一时间相同，
            通过这样保留之前的长期依赖关系。

In [1]:
# import os
# os.listdir('/home/kesci/input')

['d2lzh1981', 'houseprices2807', 'jaychou_lyrics4703', 'd2l_jay9460']

In [39]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F



In [40]:
def load_data_jay_lyrics():
    with open("C://jupyter_notebook//boyu//jaychou_lyrics.txt") as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace("\n", ' ').replace('\r', ' ').replace("'", " ").replace(" '", " ")
    idx_to_char = list(set(corpus_chars))
    char_to_idx = {char:i for i, char in enumerate(idx_to_char)}
    vocab_size = len(idx_to_char)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 初始化参数

In [41]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():  
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32) #正态分布
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
     
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

def init_gru_state(batch_size, num_hiddens, device):   #隐藏状态初始化
    return (torch.zeros((batch_size, num_hiddens), device=device), )

will use cuda


### GRU模型
$$
R_{t} = σ(X_tW_{xr} + H_{t−1}W_{hr} + b_r)\\    
Z_{t} = σ(X_tW_{xz} + H_{t−1}W_{hz} + b_z)\\  
\widetilde{H}_t = tanh(X_tW_{xh} + (R_t ⊙H_{t−1})W_{hh} + b_h)\\
H_t = Z_t⊙H_{t−1} + (1−Z_t)⊙\widetilde{H}_t
$$

In [42]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + R * torch.matmul(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

### 训练模型

In [43]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [44]:
# 在上一节定义的
import time
import math

# 将一维向量x转为one-hot向量
# 传入参数：
# x：shape: 为(shape[0]行数, shape[1]列数)的tensor
# n_class : 语料库的长度
# 输出：    
#          （x.shape[0], n_class）的one-hot tensor
def one_hot(x, n_class, dtype=torch.float32):
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)  # shape: (n, n_class)
#     print("x.long.view(-1, 1): ", x.long().view(-1, 1))
    # scatter_函数的说明：
    # 传入参数 (int dim, Tensor index, Tensor src)
    # 第一个参数指定维度，这里dim=1即为对每一行的某些索引的位置修改为src
    # 第二个参数指定需要修改的元素的索引，因为我们第一个参数指定了dim=1，因此在这里需要传入一个列向量
    #            指定相应的索引，如[0,2]就是我们要将[0, 0]位置和[1,2]位置修改为src
    # 第三个参数指定应该改为什么值
    result.scatter_(1, x.long().view(-1, 1), 1)  # result[i, x[i, 0]] = 1
    return result

def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        # l_sum 是loss的总和，用于计算perplexity
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
            # inputs是num_steps个形状为(batch_size, vocab_size)的矩阵
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            # 拼接之前outpus 是每个元素为  长度为num_steps的列表
            outputs = torch.cat(outputs, dim=0)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成形状为
            # (num_steps * batch_size,)的向量，这样跟输出的行一一对应
            y = torch.flatten(Y.T)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            SGD(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            # 输出epoch，perplexity和训练时间time
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            # 使用predict_rnn输出训练结果
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

                
# 在线性回归章节中实现的sgd
def SGD(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size
                
            
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)
                
                
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    # 必须将indices的长度clip成batch_size的倍数，否则会报RuntimeError
    # RuntimeError: shape '[batch_size, -1]' is invalid for input of size 
    
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps  # 列数 / 时间步数 = 可以采样的次数
    # 每一次产生一个抽样
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y
        
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)

    def forward(self, inputs, state):
        # inputs.shape: (batch_size, num_steps)
        X = to_onehot(inputs, vocab_size)
        X = torch.stack(X)  # X.shape: (num_steps, batch_size, vocab_size)
        hiddens, state = self.rnn(X, state)
        hiddens = hiddens.view(-1, hiddens.shape[-1])  # hiddens.shape: (num_steps * batch_size, hidden_size)
        output = self.dense(hiddens)
        return output, state
    
    
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        state = None
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state[0].detach_()
                    state[1].detach_()
                else: 
                    state.detach_()
            (output, state) = model(X, state) # output.shape: (num_steps * batch_size, vocab_size)
            y = torch.flatten(Y.T)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))
                
# 循环调用rnn(X, state, params), 并根据RNN计算步骤得到一个句子
# 
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)  # 初始化隐层变量，下面将不断循环得到新的state
    output = [char_to_idx[prefix[0]]]   # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y[0].argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])


def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]  # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])

In [21]:
train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                        vocab_size, device, corpus_indices, idx_to_char,
                        char_to_idx, False, num_epochs, num_steps, lr,
                        clipping_theta, batch_size, pred_period, pred_len,
                        prefixes)

epoch 40, perplexity 120.523370, time 2.45 sec
 - 分开 我想你你 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 
 - 不分开 我想你你 我不要你 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要
epoch 80, perplexity 19.478194, time 1.55 sec
 - 分开 我想要你 我不要再想 我不 我不 我不要再想你 我不要再想你的怒火 我想揍你已经单单 想要你的爱写
 - 不分开 我爱你的生笑 就知后起 是谁的练屋极 干什么 干什么 干什么 我想要你的生笑 干什么 干什么 干什
epoch 120, perplexity 3.007660, time 2.08 sec
 - 分开 不要再这样里 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该了好生活 后知后觉 又
 - 不分开可爱不知不要  我不要再想 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 
epoch 160, perplexity 1.206532, time 2.28 sec
 - 分开 不要再这样打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一切 让家庭回到过去甜甜 温馨的欢乐香味
 - 不分开熬熬  没有你烦 我有多烦恼  没有你烦我有多烦恼多难熬  穿过云层 我试著努力向你奔跑 爱才送到 


### 简洁实现

In [47]:
num_hiddens=256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['世人', '如雪']

lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.018914, time 0.28 sec
 - 世人耿直不屈 一身正气 他们儿子我习惯 从小就耳濡目染 什么刀枪跟棍棒 我都耍的有模有样 什么兵器最喜欢


KeyError: '雪'

# LSTM

** 长短期记忆long short-term memory **:  
遗忘门:控制上一时间步的记忆细胞 
输入门:控制当前时间步的输入  
输出门:控制从记忆细胞到隐藏状态  
记忆细胞：⼀种特殊的隐藏状态的信息的流动  


![Image Name](https://cdn.kesci.com/upload/image/q5jk2bnnej.png?imageView2/0/w/640/h/640)

$$
I_t = σ(X_tW_{xi} + H_{t−1}W_{hi} + b_i) \\
F_t = σ(X_tW_{xf} + H_{t−1}W_{hf} + b_f)\\
O_t = σ(X_tW_{xo} + H_{t−1}W_{ho} + b_o)\\
\widetilde{C}_t = tanh(X_tW_{xc} + H_{t−1}W_{hc} + b_c)\\
C_t = F_t ⊙C_{t−1} + I_t ⊙\widetilde{C}_t\\
H_t = O_t⊙tanh(C_t)
$$


### 初始化参数

In [27]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

def init_lstm_state(batch_size, num_hiddens, device):
    # 普通的RNN只有一个隐层变量构成的单元素元组
    # LSTM的state包括隐层变量和记忆细胞
    # 同时初始化隐层变量和记忆细胞初始量
    return (torch.zeros((batch_size, num_hiddens), device=device), 
            torch.zeros((batch_size, num_hiddens), device=device))

will use cuda


### LSTM模型

In [28]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

### 训练模型

In [30]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['想你', '我的']

train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 181.931566, time 1.99 sec
 - 想你 我不的我你 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的
 - 我的 我不的我你 我不的我你 我不的我你 我不的我你 我不的我你 我不的我你 我不的我你 我不的我你 我
epoch 80, perplexity 45.312586, time 1.98 sec
 - 想你 我想的这爱我 不知 我不要 我不要再想 我不要 不不 我不要 我不要 我不要 我不要 我不要 我不
 - 我的可爱 爱爱不再 我不要再想我 爱不 我不要 我不要再想你 不知 我不 我不要再想不 我不要 不不 我
epoch 120, perplexity 9.407950, time 2.05 sec
 - 想你的你样 我不要你想你就你 不不躲 你给我 你不球 是你怎么医药 说知后觉 又跟了一个秋 后知后觉 我
 - 我的可爱 我想要再想 我不要再想你 不知 你想经你你 说散 你想很久了吧? 我想你你想你  你知你的你着
epoch 160, perplexity 2.720867, time 2.03 sec
 - 想你的你怎 我不要再想我 不知不觉 你已离离开我 不知不觉 我跟了这节奏 后知后觉 又过了离开秋 后知不
 - 我的爱女 怎已经离我已单 就散开 你给我 难不球 我想再这活 不知不觉 你不了这节奏 后知后觉 又过了一


### 简洁实现

In [32]:
num_hiddens=256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['想你', '你的']

lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(lstm_layer, vocab_size)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.021333, time 0.31 sec
 - 想你想很久了吧? 我不想拆穿你 当作 是你开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑
 - 你的星如果听的见 它一定实现 它一定实现 娘子 娘子却依旧每日 折一枝杨柳 你在那里 在小村外的溪边河口
epoch 80, perplexity 1.011581, time 0.31 sec
 - 想你想的睡不著 嘴嘟嘟那可爱的模样 还有在你身上香香的味道 我的快乐是你 想你想的都会笑 没有你在 我有
 - 你的星如果听的见 它一定实现 它一定实现 娘子 娘子却依旧每日 折一枝杨柳 你在那里 在小村外的溪边河口
epoch 120, perplexity 1.013069, time 0.33 sec
 - 想你想的睡不著 嘴嘟嘟那可爱的模样 还有在你身上香香的味道 我的快乐是你 想你想的都会笑 没有你在 我有
 - 你的星如果听的见 它一定实现 它一定实现 娘子 娘子却依旧每日 折一枝杨柳 你在那里 在小村外的溪边河口
epoch 160, perplexity 1.047109, time 0.32 sec
 - 想你想的你笑 那么凶 如果真的我有一双翅膀 二双翅膀 随时出发 偷偷出发 我一定带我妈走  从前的教育别
 - 你的星如果听的见 它一定会呵护著你 也逗你笑 你对我有多重要 我后悔没让你知道 安静的听你撒娇 看你睡著


# 深度循环神经网络  

![Image Name](https://cdn.kesci.com/upload/image/q5jk3z1hvz.png?imageView2/0/w/320/h/320)


$$
\boldsymbol{H}_t^{(1)} = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(1)} + \boldsymbol{H}_{t-1}^{(1)} \boldsymbol{W}_{hh}^{(1)} + \boldsymbol{b}_h^{(1)})\\
\boldsymbol{H}_t^{(\ell)} = \phi(\boldsymbol{H}_t^{(\ell-1)} \boldsymbol{W}_{xh}^{(\ell)} + \boldsymbol{H}_{t-1}^{(\ell)} \boldsymbol{W}_{hh}^{(\ell)} + \boldsymbol{b}_h^{(\ell)})\\
\boldsymbol{O}_t = \boldsymbol{H}_t^{(L)} \boldsymbol{W}_{hq} + \boldsymbol{b}_q
$$

In [33]:


num_hiddens=256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['想你', '我想']

lr = 1e-2 # 注意调整学习率

# 隐层为2
gru_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens,num_layers=2)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.036336, time 0.42 sec
 - 想你 不想太多 我想一定是我听错弄错搞错 拜托 我想是你的脑袋有问题 随便说说 其实我早已经猜透看透不想
 - 我想 你想你 你爱我 我想就这样牵着你的手不放开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和
epoch 80, perplexity 1.019582, time 0.41 sec
 - 想你 你说的话 你甘会听 不要再这样打我妈妈 难道你手不会痛吗 不要再这样打我妈妈 我说的话 你甘会听 
 - 我想印人我痛 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著
epoch 120, perplexity 1.012502, time 0.42 sec
 - 想你 难过 是因为闷了很久 是因为想了太多 是心理起了作用 你说 苦笑常常陪着你 在一起有点勉强 该不该
 - 我想 你想的你的细汉 从小到大只有妈妈的温暖  为什么我爸爸 那么凶 如果真的我有一双翅膀 二双翅膀 随
epoch 160, perplexity 1.011946, time 0.41 sec
 - 想你 却又再考倒我 说散 你想很久了吧? 我不想拆穿你 当作 是你开的玩笑 想通 却又再考倒我 说散 你
 - 我想印你 烦有多烦恼多难熬  穿过云层 我试著努力向你奔跑 爱才送到 你却已在别人怀抱 就是开不了口让她


In [37]:
# 隐层为6
# 层数过深导致收敛困难
gru_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens,num_layers=6)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 9.039007, time 0.55 sec
 - 想你的没有你妈 不要再想 不要再这我妈你 让我开去 可过开始一场热颗在吧? 我不想再多 不要我 让在休太
 - 我想爱有我你想 不多的没有 让在考动 一小好剧 我说不能我没始不恼 我要再想你 想要我 让就考动 你在感
epoch 80, perplexity 1.141782, time 0.51 sec
 - 想你你都伤口被没拆伤口 可家透你 没有你在 在 伤熬 的子联看力中的漂窗的到默默默的在样的可以 的只没有
 - 我想印地安的传说 还真是 瞎透了 什么一个戏 他们让开始想多 娘子 可手一场热粥 配上几斤的牛肉 我说店
epoch 120, perplexity 1.028892, time 0.51 sec
 - 想你你都伤口被你拆封 誓言太沉重泪被纵容 脸上汹涌失控 穿梭时间的画面的钟 从反方向开始移动 回到当初爱
 - 我想爱说 有再三开开 瞎透了一定实一天人我漂许的心愿你一天主一宙里 你每天我一个人决恼 你没有你在 我有
epoch 160, perplexity 1.015062, time 0.52 sec
 - 想你你 爸我回情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不要再想 我不 我不 我不要再想
 - 我想爱过不有样我要拆口缓香等你等的害  说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很久 别想躲 说


# 双向循环神经网络 

![Image Name](https://cdn.kesci.com/upload/image/q5j8hmgyrz.png?imageView2/0/w/320/h/320)

$$ 
\begin{aligned} \overrightarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(f)} + \overrightarrow{\boldsymbol{H}}_{t-1} \boldsymbol{W}_{hh}^{(f)} + \boldsymbol{b}_h^{(f)})\\
\overleftarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(b)} + \overleftarrow{\boldsymbol{H}}_{t+1} \boldsymbol{W}_{hh}^{(b)} + \boldsymbol{b}_h^{(b)}) \end{aligned} $$
$$
\boldsymbol{H}_t=(\overrightarrow{\boldsymbol{H}}_{t}, \overleftarrow{\boldsymbol{H}}_t)
$$
$$
\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q
$$

In [38]:
num_hiddens=128
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e-2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['你是', '我是']

lr = 1e-2 # 注意调整学习率

# 效果不好
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens,bidirectional=True)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.001326, time 0.28 sec
 - 你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是
 - 我是我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不
epoch 80, perplexity 1.000422, time 0.31 sec
 - 你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是
 - 我是我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不
epoch 120, perplexity 1.000209, time 0.30 sec
 - 你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是
 - 我是我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不
epoch 160, perplexity 1.000124, time 0.30 sec
 - 你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是你是
 - 我是我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不我不
